In [1]:
import numpy as np
import scipy as sp

In [2]:
def rodrigez(fi,p):
    e=np.eye(3)
    px=np.array([[0,-p[2],p[1]],
                 [p[2],0,-p[0]],
                 [-p[1],p[0],0]])
    ppT=np.array([[p[0]**2,p[0]*p[1],p[0]*p[2]]
                 ,[p[1]*p[0],p[1]**2,p[1]*p[2]]
                 ,[p[2]*p[0],p[2]*p[1],p[2]**2]])
    return (1-np.cos(fi))*ppT+np.cos(fi)*e+np.sin(fi)*px

In [180]:
def Euler2A(uglovi):
    A=rodrigez(uglovi[0],[0,0,1])@rodrigez(uglovi[1],[0,1,0])@rodrigez(uglovi[2],[1,0,0])
    A = np.where(np.isclose(A, 0) , 0 , A)  # da bi izbegli -0. u rezultatu
    return A

In [179]:
def AxisAngle2A(pphi):
    A=rodrigez(pphi[3],[pphi[0],pphi[1],pphi[2]])
    A = np.where(np.isclose(A, 0) , 0.0 , A)  # izbegavanje -0. u rezultatu
    return A

In [3]:
def nadji_vektor(sopstveni_vektori):
    for i in range(len(sopstveni_vektori)):
        j=0;
        for elem in sopstveni_vektori[:,i]:
            if elem.imag != 0:
                break
            j+=1;
        if j==3:
            return sopstveni_vektori[:,i]
    raise Exception("Nema odgovarajuceg vektora!")

In [178]:
def A2AxisAngle(A):
# 1. Odredi jedinicni sopstveni vektor p za lambda=1
# 2. Odredi proizvoljan jedinicni vektor u normalan na p
# 3. Odredi u_prim koji je jednak A*u (u_prim je jedinicni)
# 4. Odredi fi koji je jednak arccos(u,u_prim)
# 5. Ako je mesoviti proizvod u,u_prim,p < 0, onda p=-p
    provera=A@A.T
    provera=np.where(np.isclose(provera,0),0,provera)
    provera=np.where(np.isclose(provera,1),1,provera)
    if (A.shape[0]!=A.shape[1]) or (not np.isclose(np.linalg.det(A),1)) or not np.all(provera==np.identity(A.shape[0])):
        return "Nije matrica kretanja!"
    if np.array_equal(A,np.eye(3)):
        return np.array([1,0,0]),0
# 1.
    _,sopstveni_vektori=sp.linalg.eig(A,left=True,right=False) # Mora da se importuje scipy.linalg umesto samo scipy
    # p=sopstveni_vektori[:,2] # Problem: Rucno biram odgovarajuci sopstveni vektor, jer nije tacno resenje sa svakim!
    p=nadji_vektor(sopstveni_vektori)
    #p=np.array(p)
    p=np.real(p)
# 2. Gram Šmit ortogonalizacija
    u = np.array([1.0,2.0,3.0])
    u -= u.dot(p) * p / np.linalg.norm(p)**2 
    u /= np.linalg.norm(u)
# 3.
    u_prim=A@u
    u_prim/=np.linalg.norm(u_prim)
# 4.    
    fi=np.arccos(u.dot(u_prim)/(np.linalg.norm(u)*np.linalg.norm(u_prim)))
# 5.
    if np.dot(u,np.cross(u_prim,p)) < 0:
        p=-p
    #return [p[0],p[1],p[2],fi]
    pphi = np.array([p[0],p[1],p[2],fi])  # osa i ugao idu u jedan vektor
    pphi = np.where(np.isclose(pphi, 0) , 0 , pphi)  # izbegavanje -0. u rezultatu
    return pphi

In [181]:
def A2Euler(A):
    uglovi=np.array([0,0,0]) # Zbog testera zadatka
    provera=A@A.T
    provera=np.where(np.isclose(provera,0),0,provera)
    provera=np.where(np.isclose(provera,1),1,provera)
    if (A.shape[0]!=A.shape[1]) or (not np.isclose(np.linalg.det(A),1)) or not np.all(provera==np.identity(A.shape[0])):
        return "Nije matrica kretanja!"
    if A[2][0] < 1:
        if A[2][0] > -1: #jedinstveno resenje
            psi=np.arctan2(A[1][0],A[0][0])
            teta=np.arcsin(-A[2][0])
            fi=np.arctan2(A[2][1],A[2][2])
            uglovi=np.array([psi,teta,fi]) #
            #return np.array([psi,teta,fi])
        else: # Ox3=-Oz
            psi=np.arctan2(-A[0][1],A[1][1])
            teta=np.pi/2
            fi=0
            #return np.array([psi,teta,fi])
            uglovi=np.array([psi,teta,fi]) #
    else: #Ox3=Oz
        psi=np.arctan2(-A[0][1],A[1][1])
        teta=-np.pi/2
        fi=0
        #return np.array([psi,teta,fi])
        uglovi=np.array([psi,teta,fi]) #
    uglovi = np.where(np.isclose(uglovi, 0) , 0 , uglovi) # Isto  
    return uglovi #